# Final Python Work - E201701 - Ugo Carotti

### Part 1: Data Extraction/Transformation

Use the command below to get the world_bank_data package:

conda install -c sclare world_bank_data or pip install -c sclare world_bank_data 

Here are the packages I used to complete the coursework.

In [1]:
import pandas as pd
import world_bank_data as wb
import numpy as np
import matplotlib.pyplot as plt

I used the keyword search in order to identify the datasets we are interested in.

In [2]:
wb.search_indicators('life expectancy at birth')

,name,unit,source,sourceNote,sourceOrganization,topics
id,,,,,,
SP.DYN.LE00.FE.IN,"Life expectancy at birth, female (years)",,World Development Indicators,Life expectancy at birth indicates the number ...,(1) United Nations Population Division. World ...,"Health ,Gender,Social Development"
SP.DYN.LE00.IN,"Life expectancy at birth, total (years)",,World Development Indicators,Life expectancy at birth indicates the number ...,(1) United Nations Population Division. World ...,Health
SP.DYN.LE00.MA.IN,"Life expectancy at birth, male (years)",,World Development Indicators,Life expectancy at birth indicates the number ...,(1) United Nations Population Division. World ...,"Health ,Gender,Social Development"
SP.DYN.LIFE.MF,Life Expectancy at Birth(years),,WDI Database Archives,,,
UNDP.HDI.XD,Human development index (HDI),,Africa Development Indicators,The Human Development Index (HDI) is a summary...,http://hdr.undp.org/en/statistics/data/,


Once identified, I extract each of them.

In [3]:
pd.set_option("display.max_rows", 10, "display.max_columns", 10) 
set1 = pd.DataFrame(wb.get_series('SP.DYN.LE00.IN', mrv=19))
set1

SP.DYN.LE00.IN
Country    Series                                  Year                
Arab World Life expectancy at birth, total (years) 2000       67.800591
                                                   2001       68.072519
                                                   2002       68.340246
                                                   2003       68.610505
                                                   2004       68.886188
...                                                                 ...
Zimbabwe   Life expectancy at birth, total (years) 2014       58.410000
                                                   2015       59.534000
                                                   2016       60.294000
                                                   2017       60.812000
                                                   2018       61.195000

[5016 rows x 1 columns]

In [4]:
set2 = pd.DataFrame(wb.get_series('SP.DYN.LE00.FE.IN', mrv=19))

In [5]:
set3 = pd.DataFrame(wb.get_series('SP.DYN.LE00.MA.IN', mrv=19))

I start merging them on Country and Year.

In [6]:
set_inter = set1.merge(set2, on=['Country','Year'])
set_inter

SP.DYN.LE00.IN  SP.DYN.LE00.FE.IN
Country    Year                                   
Arab World 2000       67.800591          69.699913
           2001       68.072519          69.959985
           2002       68.340246          70.218646
           2003       68.610505          70.483623
           2004       68.886188          70.758013
...                         ...                ...
Zimbabwe   2014       58.410000          59.892000
           2015       59.534000          60.984000
           2016       60.294000          61.719000
           2017       60.812000          62.220000
           2018       61.195000          62.598000

[5016 rows x 2 columns]

In [7]:
dataset = set_inter.merge(set3,on=['Country','Year'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 5016 entries, ('Arab World', '2000') to ('Zimbabwe', '2018')
Data columns (total 3 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   SP.DYN.LE00.IN     4679 non-null   float64
 1   SP.DYN.LE00.FE.IN  4679 non-null   float64
 2   SP.DYN.LE00.MA.IN  4679 non-null   float64
dtypes: float64(3)
memory usage: 145.2+ KB


Once I have my dataset I rename the columns to make it clearer.

In [8]:
dataset = dataset.rename(columns={'SP.DYN.LE00.IN':'Combined_LE', 'SP.DYN.LE00.FE.IN':'Female_LE', 'SP.DYN.LE00.MA.IN':'Male_LE'})
dataset

Combined_LE  Female_LE    Male_LE
Country    Year                                   
Arab World 2000    67.800591  69.699913  66.026259
           2001    68.072519  69.959985  66.308686
           2002    68.340246  70.218646  66.585351
           2003    68.610505  70.483623  66.862754
           2004    68.886188  70.758013  67.144215
...                      ...        ...        ...
Zimbabwe   2014    58.410000  59.892000  56.632000
           2015    59.534000  60.984000  57.778000
           2016    60.294000  61.719000  58.565000
           2017    60.812000  62.220000  59.105000
           2018    61.195000  62.598000  59.501000

[5016 rows x 3 columns]

I get rid of the incomplete data by running a loop over the multiIndex, checking for each country the number of null values. I divide the count by 3 as the values alwayscome by three. There are 19 years (2000 to 2018), so if I have more than 9 for the missing count, I will have less than 10 values per columns. I simply drop the row of countries without enough data.

In [9]:
for country, year in dataset.groupby(level=0): 
    count = dataset.loc[country].isnull().sum().sum()
    count = count/3
    if count > 9:
        print(country,' Not enough values')
        dataset = dataset.drop(country)

American Samoa  Not enough values
Andorra  Not enough values
British Virgin Islands  Not enough values
Cayman Islands  Not enough values
Dominica  Not enough values
Gibraltar  Not enough values
Isle of Man  Not enough values
Marshall Islands  Not enough values
Monaco  Not enough values


C:\Users\carot\anaconda3\envs\Hwork2\lib\site-packages\pandas\core\generic.py:3887: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  obj = obj._drop_axis(labels, axis, level=level, errors=errors)


Nauru  Not enough values
Northern Mariana Islands  Not enough values
Not classified  Not enough values
Palau  Not enough values
San Marino  Not enough values
Sint Maarten (Dutch part)  Not enough values
St. Kitts and Nevis  Not enough values
Turks and Caicos Islands  Not enough values
Tuvalu  Not enough values


I check the look of my dataset before continuing any further.

In [10]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4674 entries, ('Arab World', '2000') to ('Zimbabwe', '2018')
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Combined_LE  4664 non-null   float64
 1   Female_LE    4664 non-null   float64
 2   Male_LE      4664 non-null   float64
dtypes: float64(3)
memory usage: 136.2+ KB


I save my dataframe as a csv file, on my desk. This is to be changed, of course, unless using my computer.

In [11]:
dataset.to_csv(r'C:\Users\carot\OneDrive\Bureau\ds.csv')

### Part 2: Test Questions

#### Q1: How many countries/regions are in the dataset?

I can obtain this information by counting the number of countries in my index.

In [12]:
nb_countries_or_regions = len(dataset.groupby(level=0))
print('There are',nb_countries_or_regions,'countries or regions in the dataset.')

There are 246 countries or regions in the dataset.


#### Q2: Which country has the highest total life expectancy in 2018?

By only selecting the year 2018 and the appropriate column, I then ask for the maximum value in the column Combined_LE and ask for thhe full row, providing me with all the information needed.

In [13]:
data2018 = dataset.xs('2018', level=1, drop_level=False)
data2018 = data2018.drop(['Female_LE','Male_LE'],axis=1)
print(data2018.iloc[[data2018['Combined_LE'].argmax()]])

                           Combined_LE
Country              Year             
Hong Kong SAR, China 2018    84.934146


#### Q3: Which has the lowest?

Follow the same steps from Q2, but ask forr the minimum value.

In [14]:
data2018 = dataset.xs('2018', level=1, drop_level=False)
data2018 = data2018.drop(['Female_LE','Male_LE'],axis=1)
print(data2018.iloc[[data2018['Combined_LE'].argmin()]])

                               Combined_LE
Country                  Year             
Central African Republic 2018       52.805


#### Q4: Which country has the greatest absolute difference in male and female life expectancy, and what is the difference, for the latest data point?

First, create the appropriate dataset by gathering the latest data provided for each country. One country has no data for 2018 and 2017, so it is necessary to implement a system that checks for the data's latest year instead of simply taking 2018 rows.

The abs difference is calculated as soon as the year is identified.

Print the new datset to check intermediary result.

In [15]:
M_F_diff = dataset.drop(['Combined_LE'],axis=1)
abs_dif = []
for country, year in M_F_diff.groupby(level=0):
    yr = 2018
    while True:
        if np.isnan(M_F_diff.loc[(country,str(yr)), 'Female_LE']) and np.isnan(M_F_diff.loc[(country,str(yr)), 'Male_LE']):
            yr -= 1
        else:
            dif = abs(M_F_diff.loc[(country,str(yr)), 'Female_LE'] - M_F_diff.loc[(country,str(yr)), 'Male_LE'])
            abs_dif.append([country,dif])
            break
abs_dif = pd.DataFrame(abs_dif)
M_F_diff = abs_dif.rename(columns={0:'Country',1:'abs_Diff_M_F'})

M_F_diff

,Country,abs_Diff_M_F
0,Afghanistan,2.97900
1,Albania,3.35100
2,Algeria,2.44400
3,Angola,5.60200
4,Antigua and Barbuda,2.26200
...,...,...
241,West Bank and Gaza,3.33200
242,World,4.47307
243,"Yemen, Rep.",3.37700
244,Zambia,5.91400


Simply look for thhe maximum in the column abs_Diff_M_F and ask for the full row to be returned.

In [16]:
print(M_F_diff.iloc[[M_F_diff['abs_Diff_M_F'].argmax()]])

                  Country  abs_Diff_M_F
218  Syrian Arab Republic        11.185


#### Q5: Which 5 countries have shown the greatest % increase in total life expectancy, between the earliest and latest data points, and what was the increase for each?

This question requires to check for the earliest and latest data available per country. Therefore, I used the technique implemented in Q4. If the Life Expectancy is not increasing from the earliest to the latest year, then the data is ignored. Once the increase is calculated and the dataframe created, I sort the dataset from highest value to lowest and only display the 5 highest, like asked.

In [23]:
percent_life_increase = []
Q5 = dataset.drop(['Female_LE','Male_LE'],axis=1)

for country, year in Q5.groupby(level=0):
    yr_min = 2000
    yr_max = 2018
    age1 = 0
    age2 = 0
    while True:
        if np.isnan(Q5.loc[(country,str(yr_max)), 'Combined_LE']):
            yr_max -= 1
        else:
            age2 = Q5.loc[(country,str(yr_max)), 'Combined_LE']
            break
            
    while True:
        if np.isnan(Q5.loc[(country,str(yr_min)), 'Combined_LE']):
            yr_min += 1
        else:
            age1 = Q5.loc[(country,str(yr_min)), 'Combined_LE']
            break
    if age1 >= age2:
        continue
    else:
        percent = ((age2-age1)/age1)*100
        percent_life_increase.append([country, percent])
        
        
percent_life_increase = pd.DataFrame(percent_life_increase)
percent_life_increase = percent_life_increase.sort_values(by=[1], ascending=False)
percent_life_increase = percent_life_increase.rename(columns={0:'Country',1:'LE_increase_in_%'})
percent_life_increase.head(5)

,Country,LE_increase_in_%
242,Zambia,44.340909
139,Malawi,41.490353
187,Rwanda,41.215647
194,Sierra Leone,37.696813
243,Zimbabwe,37.057941


### Part 3: Analysis

Question: What is the relationship between total life expectancy in a country, and the
difference between its male & female life expectancies? 

Give one possible
explanation for the relationship.

In order to visualize the data, I first create a dataset using the results of the questions above. I then use a scatter plot to analyze the result.

In [ ]:
P3 = dataset.drop(['Female_LE','Male_LE'],axis=1)
P3 = P3.merge(M_F_diff, on = 'Country')
P3 = P3.sample(1000)
plt.scatter(P3.abs_Diff_M_F, P3.Combined_LE)